In [11]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
from scipy.stats import linregress

df = pd.read_csv("cleandata/athletes_and_medalcount_noNaN")
df1 = pd.read_csv("cleandata/athletes_and_medalcount")

In [12]:
df = df[['ID', 'Name', 'Sex', 'Age', 'Height', 'Weight', 'Team',
       'NOC', 'Games', 'Year', 'Season', 'City', 'Sport', 'Event', 'region_x',
       'notes_x', 'Bronze', 'Gold', 'Silver', 'n/a']]
df["Medals"] = df['Bronze'] + df['Silver']+ df['Gold']
df["Attempts"] = df['Bronze'] + df['Silver']+ df['Gold']+ df['n/a']

In [13]:
df1 = df1[['ID', 'Name', 'Sex', 'Age', 'Height', 'Weight', 'Team',
       'NOC', 'Games', 'Year', 'Season', 'City', 'Sport', 'Event', 'region_x',
       'notes_x', 'Bronze', 'Gold', 'Silver', 'n/a']]
df1["Medals"] = df1['Bronze'] + df1['Silver']+ df1['Gold']
df1["Attempts"] = df1['Bronze'] + df1['Silver']+ df1['Gold']+ df1['n/a']

In [14]:
sports_list = df1["Sport"].unique()
r_weight = []
r_height = []
medal_rate = []
num_of_events = []
games = []
competitors = []
weight_avg = []
height_avg = []

In [15]:
for sport in sports_list:
    try:
        df_loop = df
        df_loop = df_loop.loc[df_loop['Sport'] == f'{sport}']
        df_loop = df_loop.groupby(by = 'ID').agg({
            "Height" : "max",
            "Weight" : "max",
            "Medals" : "sum",
            "Attempts" : "sum"
        })
        try:
            df_loop["Medal Rate"] = df_loop["Medals"] / df_loop["Attempts"] *100
        except:
            0
        plt.clf()
        df_loop = df_loop.loc[df_loop["Medals"] >= 1]
        x_val = df_loop["Height"]
        y_val = df_loop["Medals"]
        (slope, intercept, rvalue, pvalue, stderr) = linregress(x_val, y_val)
        regress_values = x_val * slope + intercept
        plt.hist(x_val, bins= 25)
        plt.xlabel('Height')
        plt.ylabel('Medals')
        plt.title("Height vs Medals")
        plt.savefig(f'breakdowns/plot_outputs/{sport}xheight.jpeg')
        r_height.append(rvalue**2)
        height_slope.append(slope)
        plt.clf()
        x_val = df_loop["Weight"]
        y_val = df_loop["Medals"]
        plt.hist(x_val, bins= 25)
        plt.xlabel('Weight')
        plt.ylabel('Medals')
        plt.title("Weight vs Medals")
        plt.savefig(f'breakdowns/plot_outputs/{sport}xweight.jpeg')
        plt.clf()
    except:
        r_height.append(0)
        r_weight.append(0)

    df_loop_sport = df1
    df_loop_sport = df_loop_sport.loc[df_loop_sport['Sport'] == f'{sport}']
    df_loop_sport = df_loop_sport.agg({
                "Medals" : "sum",
                "Attempts" : "sum",
                "Event" : "nunique",
                "Games" : "nunique",
                "ID" : "nunique",
                "Height" : "mean",
                "Weight" : "mean",
            })        
    weight_avg.append(df_loop_sport["Weight"].max())
    height_avg.append(df_loop_sport["Height"].max())
    games.append(df_loop_sport["Games"])
    competitors.append(df_loop_sport["ID"])
    num_of_events.append(df_loop_sport["Event"])
    medal_rate.append(df_loop_sport["Medals"] / df_loop_sport["Attempts"] *100)

<Figure size 432x288 with 0 Axes>

In [16]:
print(len(sports_list))
print(len(height_avg))
print(len(weight_avg))
print(len(medal_rate))
print(len(num_of_events))


66
66
66
66
66


In [17]:
sport_stats = pd.DataFrame({
    "Sport" : sports_list,
    "Medal Rate" : medal_rate,
    "Number of Events" : num_of_events,
    "# of Games Appeared" : games,
    "Olympians Competed": competitors,
    "Height Avg" : height_avg,
    "Weight Avg" : weight_avg
        })
sport_stats = sport_stats.sort_values("Height Avg", ascending = False)
sport_stats.head(5)

,Sport,Medal Rate,Number of Events,# of Games Appeared,Olympians Competed,Height Avg,Weight Avg
0,Basketball,23.809524,2.0,19.0,3413.0,190.869879,85.777053
37,Volleyball,28.466510,2.0,14.0,2503.0,186.994822,78.900214
46,Beach Volleyball,12.765957,2.0,6.0,383.0,186.144954,79.089219
19,Water Polo,27.483099,2.0,27.0,2599.0,184.834648,84.566446
21,Rowing,27.796130,25.0,28.0,7687.0,184.172222,80.035863


In [18]:
sport_stats.to_csv("Sport_lvl_Stats.csv")

In [19]:
sport_stats.corr()

,Medal Rate,Number of Events,# of Games Appeared,Olympians Competed,Height Avg,Weight Avg
Medal Rate,1.000000,-0.355263,-0.548085,-0.301381,0.167255,0.366009
Number of Events,-0.355263,1.000000,0.567980,0.760457,-0.099136,-0.053754
# of Games Appeared,-0.548085,0.567980,1.000000,0.643795,-0.006727,-0.042717
Olympians Competed,-0.301381,0.760457,0.643795,1.000000,0.099971,0.035836
Height Avg,0.167255,-0.099136,-0.006727,0.099971,1.000000,0.762238
Weight Avg,0.366009,-0.053754,-0.042717,0.035836,0.762238,1.000000
